In [1]:
import numpy as np
import ee
import io
from google.api_core import exceptions, retry
import google.auth
from models import *

PROJECT = "pc530-fao-fra-rss"  # change to your cloud project name
# ee.Initialize(project=PROJECT)

## INIT WITH HIGH VOLUME ENDPOINT
credentials, _ = google.auth.default()
ee.Initialize(
credentials,
project=PROJECT,
opt_url="https://earthengine-highvolume.googleapis.com",
)

In [2]:
def get_ee_img(coords):
    """retrieve s2 image composite from ee at given coordinates. coords is a tuple of (lon, lat) in degrees."""
    ## MAKE S2 COMPOSITE IN HEXAGONS ##########################################
    # Using Cloud Score + for cloud/cloud-shadow masking
    # Harmonized Sentinel-2 Level 2A collection.
    s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

    # Cloud Score+ image collection. Note Cloud Score+ is produced from Sentinel-2
    # Level 1C data and can be applied to either L1C or L2A collections.
    csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

    # Use 'cs' or 'cs_cdf', depending on your use case; see docs for guidance.
    QA_BAND = "cs_cdf"

    # The threshold for masking; values between 0.50 and 0.65 generally work well.
    # Higher values will remove thin clouds, haze & cirrus shadows.
    CLEAR_THRESHOLD = 0.50

    # Make a clear median composite.
    sampleImage = (
        s2.filterDate("2023-01-01", "2023-12-31")
        .filterBounds(ee.Geometry.Point(coords[0], coords[1]).buffer(64*10)) # only images touching 64 pixel centroid buffer
        .linkCollection(csPlus, [QA_BAND])
        .map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))
        .median()
        .select(["B4", "B3", "B2", "B8"], ["R", "G", "B", "N"])
    )
    return sampleImage

@retry.Retry()
def get_patch(coords, image, format="NPY"):
    """Uses ee.data.ComputePixels() to get a 32x32 patch centered on the coordinates, as a numpy array."""
    
    # Output resolution in meters.
    SCALE = 10

    # Pre-compute a geographic coordinate system.
    proj = ee.Projection("EPSG:4326").atScale(SCALE).getInfo()

    # Get scales in degrees out of the transform.
    SCALE_X = proj["transform"][0]
    SCALE_Y = -proj["transform"][4]

    # Patch size in pixels.
    PATCH_SIZE = 32

    # Offset to the upper left corner.
    OFFSET_X = -SCALE_X * PATCH_SIZE / 2
    OFFSET_Y = -SCALE_Y * PATCH_SIZE / 2
    
    REQUEST = {
        "fileFormat": "NPY",
        "grid": {
            "dimensions": {"width": PATCH_SIZE, "height": PATCH_SIZE},
            "affineTransform": {
                "scaleX": SCALE_X,
                "shearX": 0,
                "shearY": 0,
                "scaleY": SCALE_Y,
            },
            "crsCode": proj["crs"],
        },
    }
    
    request = dict(REQUEST)
    request["fileFormat"] = format
    request["expression"] = image
    request["grid"]["affineTransform"]["translateX"] = coords[0] + OFFSET_X
    request["grid"]["affineTransform"]["translateY"] = coords[1] + OFFSET_Y
    return np.load(io.BytesIO(ee.data.computePixels(request)))

In [3]:
id, latlon = 1233804841, [102.56,-1.18]#[102.19,-1.54]#[-60.25204,3.86655]#[-172.3490007781034,-13.523357265222518] #[-257.82, -1.54]#
print(latlon)
image = get_ee_img(latlon)
patch = get_patch(latlon, image)
# print(patch)

[102.56, -1.18]


In [4]:
import geemap
Map = geemap.Map()
Map.addLayer(image, {"bands": ["R", "G", "B"], "min": 0, "max": 2000}, "S2")
Map.addLayer(ee.Geometry.Point(latlon), {"color": "red"}, "Centroid")
Map.centerObject(ee.Geometry.Point(latlon), 18)
Map

Map(center=[-1.18, 102.56000000000002], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…

In [5]:
import tensorflow as tf
from numpy.lib.recfunctions import structured_to_unstructured
unstruct = structured_to_unstructured(patch)
# print(unstruct)

In [6]:
rescaled = unstruct.astype(np.float64) / 10000
# print(rescaled)


In [7]:
transposed = np.transpose(rescaled, (1, 2, 0))
# print(transposed)

In [8]:
# how to get numpy array of 4 bands into correct shape for model prediction
reshaped = np.reshape(transposed, (1, 32, 32, 4))
# print(reshaped)

In [9]:
# 5-epoch resnet trained on full tfrecord set (tfrecords/all)
model_name = "resnet"
optimizer = "adam"
loss_function = "binary_crossentropy"
checkpoint = "C:\\fao-models\\saved_models\\resnet-epochs5-batch64-lr001-seed5-lrdecay5-tfrecords-all\\best_model.h5"
# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
freeze(model)

# print(model.summary())
prediction = model(reshaped)
print(prediction)

Model found: resnet
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 4)]       0         
                                                                 
 resnet50 (Functional)       (None, 1, 1, 2048)        23590848  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 24,115,649
Trainable params: 0
Non-trainable params: 24,115,649
_________________________________________________________________
None
tf.Tensor([[0.01512885]]

In [10]:
# 30-epoch resnet with 86% binary accuracy
model_name = "resnet"
optimizer = "adam"
loss_function = "binary_crossentropy"
checkpoint = "C:\\fao-models\\saved_models\\resnet-epochs30-batch64-lr001\\best_model.h5"

# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
freeze(model)
# print(model.summary())
# apply sigmoid fn to from logits to prob
prediction = model(reshaped)
print(prediction)

Model found: resnet
tf.Tensor([[0.00071617]], shape=(1, 1), dtype=float32)


In [11]:
# 10 epoch mobilenetv3small with 82% acc
model_name = "mobilenet_v3small"
optimizer = "adam"
loss_function = "binary_crossentropy"
checkpoint = "C:\\fao-models\\saved_models\\mobilenetv3small-epochs10-batch32-lr01\\best_model.h5"

# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
freeze(model)

# print(model.summary())
prediction = model(reshaped)
print(prediction)

Model found: mobilenet_v3small
tf.Tensor([[0.00288773]], shape=(1, 1), dtype=float32)


In [12]:
# 15 epoch resnet with 98% binary accuracy 
model_name = "resnet"
optimizer = "adam"
loss_function = "binary_crossentropy"
checkpoint = "C:\\fao-models\\saved_models\\resnet-epochs5-batch64-lr001-seed5-lrdecay5\\best_model.h5"
# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
freeze(model)
# print(model.summary())
prediction = model(reshaped)
print(prediction)


Model found: resnet
tf.Tensor([[0.03912623]], shape=(1, 1), dtype=float32)


In [13]:
# very early model.. didn't even save its training params
model_name = "mobilenet_v3small"
optimizer = "adam"
loss_function = "binary_crossentropy"

checkpoint = "C:\\fao-models\\saved_models\\mobilenet_v3small_batch255\\best_model.h5"

# load several model versions into memory..
model = get_model(model_name, optimizer=optimizer, loss_fn=loss_function)
model.load_weights(checkpoint)
freeze(model)
# print(model.summary())
prediction = model(reshaped)
print(prediction)

Model found: mobilenet_v3small
tf.Tensor([[0.9805746]], shape=(1, 1), dtype=float32)
